In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuração para visualização
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.float_format', '{:.2f}'.format)

# 1. LEITURA E INSPEÇÃO INICIAL
print("=" * 60)
print("1. LEITURA E INSPEÇÃO INICIAL DA BASE DE DADOS")
print("=" * 60)

# Carregar o dataset
df = pd.read_csv('atividade3_dataset.csv', sep=';', decimal=',')

print(f"Shape inicial do dataset: {df.shape}")
print(f"\nPrimeiras 5 linhas:")
print(df.head())

print(f"\nInformações sobre os tipos de dados:")
print(df.info())

print(f"\nEstatísticas descritivas:")
print(df.describe())

print(f"\nValores nulos por coluna:")
print(df.isnull().sum())

print(f"\nValores duplicados: {df.duplicated().sum()}")

# 2. TRATAMENTO DE VALORES AUSENTES
print("\n" + "=" * 60)
print("2. TRATAMENTO DE VALORES AUSENTES")
print("=" * 60)

print("Estratégias de tratamento:")
print("- ID: Sem valores nulos, mantido como está")
print("- Nome: Sem valores nulos, mantido")
print("- Idade: 6 valores nulos - preencher com mediana (menos sensível a outliers)")
print("- Cidade: Sem valores nulos, mas precisa de padronização")
print("- Produto: Sem valores nulos, mantido")
print("- Preço: 11 valores nulos - preencher com mediana por produto")
print("- Quantidade: Sem valores nulos, mantido")
print("- Data_Compra: Sem valores nulos, mas precisa converter para datetime")

# Preencher idade com mediana
idade_mediana = df['Idade'].median()
df['Idade'] = df['Idade'].fillna(idade_mediana)
print(f"\n✓ Idades nulas preenchidas com mediana: {idade_mediana:.1f} anos")

# Preencher preço com mediana por produto
preco_por_produto = df.groupby('Produto')['Preço'].transform('median')
df['Preço'] = df['Preço'].fillna(preco_por_produto)
print("✓ Preços nulos preenchidos com mediana por produto")

# 3. DETECÇÃO E REMOÇÃO DE DUPLICADOS
print("\n" + "=" * 60)
print("3. REMOÇÃO DE REGISTROS DUPLICADOS")
print("=" * 60)

duplicados_antes = df.duplicated().sum()
print(f"Registros duplicados encontrados: {duplicados_antes}")

# Remover duplicados mantendo a primeira ocorrência
df = df.drop_duplicates()
print(f"✓ Duplicados removidos: {duplicados_antes}")
print(f"Shape após remoção: {df.shape}")

# 4. CORREÇÕES E PADRONIZAÇÃO
print("\n" + "=" * 60)
print("4. PADRONIZAÇÃO E CORREÇÕES")
print("=" * 60)

# Padronizar nomes das cidades (primeira letra maiúscula)
df['Cidade'] = df['Cidade'].str.upper().str.title()
print("✓ Cidades padronizadas (formato título)")

# Converter tipos de dados
df['Idade'] = df['Idade'].astype(int)
df['Data_Compra'] = pd.to_datetime(df['Data_Compra'], format='%d-%m-%Y', errors='coerce')
print("✓ Tipos de dados convertidos:")
print("  - Idade: float → int")
print("  - Data_Compra: string → datetime")

# Verificar valores extremos em Preço
Q1 = df['Preço'].quantile(0.25)
Q3 = df['Preço'].quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR
outliers = df[df['Preço'] > limite_superior]
print(f"✓ Possíveis outliers em Preço (> R$ {limite_superior:.2f}): {len(outliers)}")
print("  Observação: Outliers mantidos para análise posterior")

# 5. EXPLORAÇÃO INICIAL APÓS LIMPEZA
print("\n" + "=" * 60)
print("5. ANÁLISE EXPLORATÓRIA APÓS LIMPEZA")
print("=" * 60)

print(f"Shape final: {df.shape}")
print(f"\nValores nulos após tratamento:")
print(df.isnull().sum())

print(f"\nEstatísticas descritivas finais:")
print(df[['Idade', 'Preço', 'Quantidade']].describe())

print(f"\nTipos de dados finais:")
print(df.dtypes)

# 6. CRIAÇÃO DE NOVAS COLUNAS (DESAFIO)
print("\n" + "=" * 60)
print("6. CRIAÇÃO DE NOVAS COLUNAS DERIVADAS")
print("=" * 60)

# Criar faixa etária
def categorizar_idade(idade):
    if idade < 25:
        return 'Jovem (18-24)'
    elif idade < 35:
        return 'Adulto Jovem (25-34)'
    elif idade < 45:
        return 'Adulto (35-44)'
    elif idade < 55:
        return 'Meia Idade (45-54)'
    elif idade < 65:
        return 'Senior (55-64)'
    else:
        return 'Idoso (65+)'

df['Faixa_Etaria'] = df['Idade'].apply(categorizar_idade)

# Criar valor total da compra
df['Valor_Total'] = df['Preço'] * df['Quantidade']

# Criar mês e ano da compra
df['Mes_Compra'] = df['Data_Compra'].dt.month
df['Ano_Compra'] = df['Data_Compra'].dt.year

print("✓ Novas colunas criadas:")
print("  - Faixa_Etaria: Categorização por faixa etária")
print("  - Valor_Total: Preço × Quantidade (valor total da venda)")
print("  - Mes_Compra: Mês extraído da data de compra")
print("  - Ano_Compra: Ano extraído da data de compra")

# 7. VISUALIZAÇÕES E ANÁLISES
print("\n" + "=" * 60)
print("7. VISUALIZAÇÕES GRÁFICAS")
print("=" * 60)

# Configuração dos gráficos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Distribuição de Idades
axes[0, 0].hist(df['Idade'], bins=15, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribuição de Idades dos Clientes', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Idade')
axes[0, 0].set_ylabel('Frequência')
axes[0, 0].grid(axis='y', alpha=0.3)

# Gráfico 2: Distribuição de Preços
axes[0, 1].hist(df['Preço'], bins=30, edgecolor='black', alpha=0.7, color='lightgreen')
axes[0, 1].set_title('Distribuição de Preços dos Produtos', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Preço (R$)')
axes[0, 1].set_ylabel('Frequência')
axes[0, 1].grid(axis='y', alpha=0.3)

# Gráfico 3: Top 5 cidades por número de vendas
top_cidades = df['Cidade'].value_counts().head(5)
axes[1, 0].bar(top_cidades.index, top_cidades.values, color='orange', alpha=0.7)
axes[1, 0].set_title('Top 5 Cidades por Número de Vendas', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Cidade')
axes[1, 0].set_ylabel('Número de Vendas')
axes[1, 0].tick_params(axis='x', rotation=45)

# Gráfico 4: Matriz de correlação
numeric_cols = ['Idade', 'Preço', 'Quantidade', 'Valor_Total']
correlation_matrix = df[numeric_cols].corr()
im = axes[1, 1].imshow(correlation_matrix, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
axes[1, 1].set_xticks(range(len(numeric_cols)))
axes[1, 1].set_yticks(range(len(numeric_cols)))
axes[1, 1].set_xticklabels(numeric_cols)
axes[1, 1].set_yticklabels(numeric_cols)
axes[1, 1].set_title('Matriz de Correlação', fontsize=14, fontweight='bold')

# Adicionar valores na matriz de correlação
for i in range(len(numeric_cols)):
    for j in range(len(numeric_cols)):
        text = axes[1, 1].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                              ha="center", va="center", color="black")

plt.tight_layout()
plt.savefig('analise_visualizacoes.png', dpi=300, bbox_inches='tight')
plt.show()

# Gráfico adicional: Distribuição de produtos
plt.figure(figsize=(10, 6))
df['Produto'].value_counts().plot(kind='bar', color='purple', alpha=0.7)
plt.title('Distribuição de Produtos Vendidos', fontsize=14, fontweight='bold')
plt.xlabel('Produto')
plt.ylabel('Quantidade Vendida')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('distribuicao_produtos.png', dpi=300, bbox_inches='tight')
plt.show()

# 8. SALVAR BASE LIMPA
print("\n" + "=" * 60)
print("8. SALVAMENTO DA BASE LIMPA")
print("=" * 60)

# Salvar base limpa
df.to_csv('base_limpa.csv', index=False, sep=';', decimal=',')
print("✓ Base limpa salva como 'base_limpa.csv'")

# 9. RELATÓRIO FINAL
print("\n" + "=" * 60)
print("RELATÓRIO FINAL DA ATIVIDADE")
print("=" * 60)

print("RESUMO DAS AÇÕES REALIZADAS:")
print("✓ Valores nulos tratados:")
print(f"  - Idade: 6 valores preenchidos com mediana ({idade_mediana:.1f} anos)")
print("  - Preço: 11 valores preenchidos com mediana por produto")

print("✓ Dados duplicados removidos: 2 registros")

print("✓ Padronizações realizadas:")
print("  - Cidades: Formato uniforme (Primeira Letra Maiúscula)")
print("  - Tipos de dados: Idade (int) e Data_Compra (datetime)")

print("✓ Novas colunas criadas:")
print("  - Faixa_Etaria: Categorização etária para análise segmentada")
print("  - Valor_Total: Valor total de cada venda (Preço × Quantidade)")
print("  - Mes_Compra e Ano_Compra: Para análise temporal")

print("\nPRINCIPAIS INSIGHTS:")
print(f"  - Dataset final: {df.shape[0]} registros válidos")
print(f"  - {df['Cidade'].nunique()} cidades diferentes representadas")
print(f"  - {df['Produto'].nunique()} tipos de produtos comercializados")
print(f"  - Período das compras: {df['Data_Compra'].min().strftime('%d/%m/%Y')} a {df['Data_Compra'].max().strftime('%d/%m/%Y')}")
print(f"  - Faixa etária predominante: {df['Faixa_Etaria'].mode().values[0]}")
print(f"  - Produto mais vendido: {df['Produto'].mode().values[0]}")

print("\nARQUIVOS GERADOS:")
print("  - base_limpa.csv: Dataset tratado e pronto para análise")
print("  - analise_visualizacoes.png: Gráficos de distribuição e correlação")
print("  - distribuicao_produtos.png: Análise de produtos mais vendidos")

print("\n" + "=" * 60)
print("BASE DE DADOS PRONTA PARA ANÁLISES FUTURAS!")
print("=" * 60)